In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
import os, glob
import numpy as np
from functools import reduce
import operator
import math
from math import pi

def getDistance(latA, lonA, latB, lonB):
    ra = 6378136.49  # radius of equator: meter
    rb = 6356755  # radius of polar: meter

    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
 
    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    if radLonA == radLonB:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(0.001))
    else:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB))**2 / math.cos(x / 2)**2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB))**2 / math.sin(x / 2)**2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)/1000
    
    dLon = radLonB - radLonA
    yy = math.sin(dLon) * math.cos(radLatB)
    xx = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = math.degrees(math.atan2(yy, xx))
    brng = (brng + 180) % 360
#     if brng>180:
#         brng=brng-360
#     brng=brng*math.pi/180
    return distance,brng

In [3]:
data_track=pd.read_csv(r'G:/小论文/data/IBTrACS_fore72.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i) 
data_track=data_track.drop(columns=['name','date'])

In [98]:
def shift(data,forcast_hour):
    cols=[]
    lat_data=data['lat']
    lon_data=data['lon']
    m=int(forcast_hour/3)+8
    n=int(forcast_hour/3)-1
    for i in range(m,n,-1):
        cols.append(data.shift(i))
    for i in range(n,-1,-1):
        cols.append(lat_data.shift(i))
    for i in range(n,-1,-1):
        cols.append(lon_data.shift(i))        
    agg = pd.concat(cols, axis=1)
    essemble=agg.dropna(axis=0,how='any')
    return essemble

In [99]:
def dis_direct(data,forecast_hour):
    dif_essemble=[]
    point=8
    for i in range(len(data)):
        x_essemble=[]
        for j in range(0,8,1):
            lon0=data[i][point*6+1]
            lat0=data[i][point*6]
            lon1=data[i][j*6+1]
            lat1=data[i][j*6]
            lat2=data[i][(j+1)*6]
            lon2=data[i][(j+1)*6+1]
            ws=data[i][(j+1)*6+2]
            p=data[i][(j+1)*6+3]
            speed=data[i][(j+1)*6+4]
            direct=data[i][(j+1)*6+5]
            distance,brng=getDistance(lat1, lon1, lat0, lon0)                                        
            f2=math.sin(lat2*pi/180)
            x_essemble.append(lat2)  #0 
            x_essemble.append(lon2)#1
            x_essemble.append(ws)#2
            x_essemble.append(p)#3
#             x_essemble.append(distance)#4
            x_essemble.append(brng)#5
#             x_essemble.append(speed)#6
#             x_essemble.append(direct)#7
            x_essemble.append(f2)#8
            dif_lat=data[i][point*6]-data[i][j*6]
            dif_lon=data[i][point*6+1]-data[i][j*6+1]
            dif_direct=data[i][point*6+5]-data[i][j*6+5]
            dif_speed=data[i][point*6+4]-data[i][j*6+4]
            dif_p=data[i][point*6+3]-data[i][j*6+3]
            dif_ws=data[i][point*6+2]-data[i][j*6+2]
            x,brngx=getDistance(lat1, lon1, lat1, lon0)
            y,brngy=getDistance(lat1, lon1, lat0, lon1)
            xx,brngxx=getDistance(lat1, lon1, lat1, lon2)
            yy,brngyy=getDistance(lat1, lon1, lat2, lon1)
            if dif_lon<=0:
                x=x
            else:
                x=-x
            if dif_lat>=0:
                y=y
            else:
                y=-y
            if lon2<=lon1:
                u=xx/360
            else:
                u=0-xx/360
            if lat2>=lat1:
                v=yy/360
            else:
                v=0-yy/360  
#             x_essemble.append(dif_lat)#9
            x_essemble.append(dif_lon)#10
            x_essemble.append(x)#11
            x_essemble.append(y)#12
#             x_essemble.append(u)#13
#             x_essemble.append(v)#14
#             x_essemble.append(dif_ws)#15
#             x_essemble.append(dif_p)#16
            x_essemble.append(dif_speed)#17 
            x_essemble.append(dif_direct)#18
        fore_point=int(forcast_hour/3)+1
        for j in range(1,fore_point,2):
            dif_lat=data[i][54+j]-data[i][point*6]
            dif_lon=data[i][54+j+fore_point-1]-data[i][point*6+1]
            x_essemble.append(dif_lat)#9
            x_essemble.append(dif_lon)#10
        x_essemble.append(lat0)
        x_essemble.append(lon0)
        dif_essemble.append(x_essemble)
    return dif_essemble

In [100]:
essemble_test=[]
for i in range(len(index_i)):
    m=index_i[i]+1       
    if i ==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    data=data_track[m:n]
    dataframe_track=pd.DataFrame(data)
    essemble=shift(dataframe_track,72)
    essemble_values=essemble.values
    essemble_list=essemble_values.tolist()
    dif_data=dis_direct(essemble_list,72)
    essemble_test.append(dif_data)    
essemble_ds=reduce(operator.add, essemble_test)

In [ ]:
import csv 
def essemble_path(name,result):
    path='./'+name
    with open(path, 'w', newline='') as csvfile:
        writer  = csv.writer(csvfile)
        for row in result:
            writer.writerow(row)
essemble_path('fore72_IBTrACS_track.csv',essemble_ds)